# Off-Policy Learning in Two-stage Recommender Systems

Many real-world recommender systems need to be highly scalable: matching millions of items with billions of users, with milliseconds latency. The scalability requirement has led to widely used two-stage recommender systems, consisting of efficient candidate generation model(s) in the first stage and a more powerful ranking model in the second stage.

Logged user feedback, e.g., user clicks or dwell time, are often used to build both candidate generation and ranking models for recommender systems. While it’s easy to collect large amount of such data, they are inherently biased because the feedback can only be observed on items recommended by the previous systems. Recently, off-policy correction on such biases have attracted increasing interest in the field of recommender system research. However, most existing work either assumed that the recommender system is a single-stage system or only studied how to apply off-policy correction to the candidate generation stage of the system without explicitly considering the interactions between the two stages.

In this work, we propose a two-stage off-policy policy gradient method, and showcase that ignoring the interaction between the two stages leads to a sub-optimal policy in two-stage recommender systems. The proposed method explicitly takes into account the ranking model when training the candidate generation model, which helps improve the performance of the whole system. We conduct experiments on real-world datasets with large item space and demonstrate the effectiveness of our proposed method.

## Pseudo code

![](https://github.com/recohut/drl-recsys/raw/a8a38a68e3606557f3501c8adc85fc6aa5726bc1/docs/_images/T257798_1.png)

## Model structure

![](https://github.com/recohut/drl-recsys/raw/a8a38a68e3606557f3501c8adc85fc6aa5726bc1/docs/_images/T257798_2.png)

![](https://github.com/recohut/drl-recsys/raw/a8a38a68e3606557f3501c8adc85fc6aa5726bc1/docs/_images/T257798_3.png)

## Training model

1. The simulation model - divides MovieLens-1M into training set, validation set and test set at 3:1:1.
2. Behavior strategy model and ranking model - Use 10,000 user-item pairs randomly generated by the simulation model to train the behavior strategy model, and then obtain a bandit data set by sampling the top-5 items of each user predicted by the behavior strategy model. Train the ranking model based on this data set, divide 2000 users as the verification set, and 4000 users as the test set.
3. Candidate generation model - Set the optimizer to AdaGrad, the initial learning rate is 0.05, and the weight limit parameters of 1-IPS and 2-IPS are set c1 = 10, c2 = 0.01 c_1 = 10, c_2=0.01.
For each training method, we trained 20 candidate generation models initialized with different random seeds. The early stopping method is applied in both one-stage evaluation and two-stage evaluation.

## Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

## Model

In [ ]:
NUM_ITEMS = 3883
NUM_YEARS = 81
NUM_GENRES = 18
NUM_USERS = 6040
NUM_OCCUPS = 21
NUM_AGES = 7
NUM_ZIPS = 3439

In [ ]:
class ItemRep(nn.Module):
    """Item representation layer."""

    def __init__(self, item_emb_size=10, year_emb_size=5, genre_hidden=5):
        super(ItemRep, self).__init__()
        self.item_embedding = nn.Embedding(
            NUM_ITEMS + 1, item_emb_size, padding_idx=0)
        self.year_embedding = nn.Embedding(NUM_YEARS, year_emb_size)
        self.genre_linear = nn.Linear(NUM_GENRES, genre_hidden)
        self.rep_dim = item_emb_size + year_emb_size + genre_hidden

    def forward(self, categorical_feats, real_feats):
        out = torch.cat(
            [
                self.item_embedding(categorical_feats[:, 0]),
                self.year_embedding(categorical_feats[:, 1]),
                self.genre_linear(real_feats)
            ],
            dim=1)
        return out

In [ ]:
class UserRep(nn.Module):
    """User representation layer."""

    def __init__(self, user_emb_size=10, feature_emb_size=5):
        super(UserRep, self).__init__()
        self.user_embedding = nn.Embedding(
            NUM_USERS + 1, user_emb_size, padding_idx=0)
        self.gender_embedding = nn.Embedding(2, feature_emb_size)
        self.age_embedding = nn.Embedding(NUM_AGES, feature_emb_size)
        self.occup_embedding = nn.Embedding(NUM_OCCUPS, feature_emb_size)
        self.zip_embedding = nn.Embedding(NUM_ZIPS, feature_emb_size)
        self.rep_dim = user_emb_size + feature_emb_size * 4

    def forward(self, categorical_feats, real_feats=None):
        reps = [
            self.user_embedding(categorical_feats[:, 0]),
            self.gender_embedding(categorical_feats[:, 1]),
            self.age_embedding(categorical_feats[:, 2]),
            self.occup_embedding(categorical_feats[:, 3]),
            self.zip_embedding(categorical_feats[:, 4])
        ]
        out = torch.cat(reps, dim=1)
        return out

In [ ]:
class ImpressionSimulator(nn.Module):
    """Simulator model that predicts the outcome of impression."""

    def __init__(self, hidden=100, use_impression_feats=False):
        super(ImpressionSimulator, self).__init__()
        self.user_rep = UserRep()
        self.item_rep = ItemRep()
        self.use_impression_feats = use_impression_feats
        input_dim = self.user_rep.rep_dim + self.item_rep.rep_dim
        if use_impression_feats:
            input_dim += 1
        self.linear = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(), nn.Linear(hidden, 50), nn.ReLU(), nn.Linear(50, 1))

    def forward(self, user_feats, item_feats, impression_feats=None):
        users = self.user_rep(**user_feats)
        items = self.item_rep(**item_feats)
        inputs = torch.cat([users, items], dim=1)
        if self.use_impression_feats:
            inputs = torch.cat([inputs, impression_feats["real_feats"]], dim=1)
        return self.linear(inputs).squeeze()

In [ ]:
class Nominator(nn.Module):
    """Two tower nominator model."""

    def __init__(self):
        super(Nominator, self).__init__()
        self.item_rep = ItemRep()
        self.user_rep = UserRep()
        self.linear = nn.Linear(self.user_rep.rep_dim, self.item_rep.rep_dim)
        self.binary = True

    def forward(self, user_feats, item_feats):
        users = self.linear(F.relu(self.user_rep(**user_feats)))
        users = torch.unsqueeze(users, 2)  # (b, h) -> (b, h, 1)
        items = self.item_rep(**item_feats)
        if self.binary:
            items = torch.unsqueeze(items, 1)  # (b, h) -> (b, 1, h)
        else:
            items = torch.unsqueeze(items, 0).expand(users.size(0), -1,
                                                     -1)  # (c, h) -> (b, c, h)
        logits = torch.bmm(items, users).squeeze()
        return logits

    def set_binary(self, binary=True):
        self.binary = binary

In [ ]:
class Ranker(nn.Module):
    """Ranker model."""

    def __init__(self):
        super(Ranker, self).__init__()
        self.item_rep = ItemRep()
        self.user_rep = UserRep()
        self.linear = nn.Linear(self.user_rep.rep_dim + 1,
                                self.item_rep.rep_dim)
        self.binary = True

    def forward(self, user_feats, item_feats, impression_feats):
        users = self.user_rep(**user_feats)
        context_users = torch.cat(
            [users, impression_feats["real_feats"]], dim=1)
        context_users = self.linear(context_users)
        context_users = torch.unsqueeze(context_users,
                                        2)  # (b, h) -> (b, h, 1)
        items = self.item_rep(**item_feats)
        if self.binary:
            items = torch.unsqueeze(items, 1)  # (b, h) -> (b, 1, h)
        else:
            items = torch.unsqueeze(items, 0).expand(
                users.size(0), -1, -1)  # (c, h) -> (b, c, h), c=#items
        logits = torch.bmm(items, context_users).squeeze()
        return logits

    def set_binary(self, binary=True):
        self.binary = binary

## Download data

In [ ]:
!wget -q --show-progress https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

ml-1m.zip           100%[===================>]   5.64M  32.7MB/s    in 0.2s    
Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


## Dataset

Create a torch dataset class for ML-1M; convert the label into binary labels (positive if rating > 3).

In [ ]:
class MovieLensDataset(Dataset):
    def __init__(self, filepath, device="cuda:0"):
        self.device = device
        ratings, users, items = self.load_data(filepath)

        self.user_feats = {}
        self.item_feats = {}
        self.impression_feats = {}

        self.user_feats["categorical_feats"] = torch.LongTensor(
            users.values).to(device)
        self.item_feats["categorical_feats"] = torch.LongTensor(
            items.values[:, :2]).to(device)
        self.item_feats["real_feats"] = torch.FloatTensor(
            items.values[:, 2:]).to(device)
        self.impression_feats["user_ids"] = torch.LongTensor(
            ratings.values[:, 0]).to(device)
        self.impression_feats["item_ids"] = torch.LongTensor(
            ratings.values[:, 1]).to(device)
        self.impression_feats["real_feats"] = torch.FloatTensor(
            ratings.values[:, 3]).view(-1, 1).to(device)
        self.impression_feats["labels"] = torch.FloatTensor(
            ratings.values[:, 2]).to(device)

    def __len__(self):
        return len(self.impression_feats["user_ids"])

    def __getitem__(self, idx):
        labels = self.impression_feats["labels"][idx]
        feats = {}
        feats["impression_feats"] = {}
        feats["impression_feats"]["real_feats"] = self.impression_feats[
            "real_feats"][idx]
        user_id = self.impression_feats["user_ids"][idx]
        item_id = self.impression_feats["item_ids"][idx]
        feats["user_feats"] = {
            key: value[user_id - 1]
            for key, value in self.user_feats.items()
        }
        feats["item_feats"] = {
            key: value[item_id - 1]
            for key, value in self.item_feats.items()
        }
        return feats, labels

    def load_data(self, filepath):
        names = "UserID::MovieID::Rating::Timestamp".split("::")
        ratings = pd.read_csv(
            os.path.join(filepath, "ratings.dat"),
            sep="::",
            names=names,
            engine="python")
        ratings["Rating"] = (ratings["Rating"] > 3).astype(int)
        ratings["Timestamp"] = (
            ratings["Timestamp"] - ratings["Timestamp"].min()
        ) / float(ratings["Timestamp"].max() - ratings["Timestamp"].min())

        names = "UserID::Gender::Age::Occupation::Zip-code".split("::")
        users = pd.read_csv(
            os.path.join(filepath, "users.dat"),
            sep="::",
            names=names,
            engine="python")
        for i in range(1, users.shape[1]):
            users.iloc[:, i] = pd.factorize(users.iloc[:, i])[0]

        names = "MovieID::Title::Genres".split("::")
        Genres = [
            "Action", "Adventure", "Animation", "Children's", "Comedy",
            "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
            "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War",
            "Western"
        ]
        movies = pd.read_csv(
            os.path.join(filepath, "movies.dat"),
            sep="::",
            names=names,
            engine="python")
        movies["Year"] = movies["Title"].apply(lambda x: x[-5:-1])
        for genre in Genres:
            movies[genre] = movies["Genres"].apply(lambda x: genre in x)
        movies.iloc[:, 3] = pd.factorize(movies.iloc[:, 3])[0]
        movies.iloc[:, 4:] = movies.iloc[:, 4:].astype(float)
        movies = movies.loc[:, ["MovieID", "Year"] + Genres]
        movies.iloc[:, 2:] = movies.iloc[:, 2:].div(
            movies.iloc[:, 2:].sum(axis=1), axis=0)

        movie_id_map = {}
        for i in range(movies.shape[0]):
            movie_id_map[movies.loc[i, "MovieID"]] = i + 1

        movies["MovieID"] = movies["MovieID"].apply(lambda x: movie_id_map[x])
        ratings["MovieID"] = ratings["MovieID"].apply(
            lambda x: movie_id_map[x])

        self.NUM_ITEMS = len(movies.MovieID.unique())
        self.NUM_YEARS = len(movies.Year.unique())
        self.NUM_GENRES = movies.shape[1] - 2

        self.NUM_USERS = len(users.UserID.unique())
        self.NUM_OCCUPS = len(users.Occupation.unique())
        self.NUM_AGES = len(users.Age.unique())
        self.NUM_ZIPS = len(users["Zip-code"].unique())

        return ratings, users, movies

In [ ]:
class SyntheticMovieLensDataset(Dataset):
    def __init__(self, filepath, simulator_path, synthetic_data_path, cut=0.764506,
                 device="cuda:0"):
        self.device = device
        self.cut = cut
        self.simulator = None

        ratings, users, items = self.load_data(filepath)

        self.user_feats = {}
        self.item_feats = {}
        self.impression_feats = {}

        self.user_feats["categorical_feats"] = torch.LongTensor(users.values)
        self.item_feats["categorical_feats"] = torch.LongTensor(
            items.values[:, :2])
        self.item_feats["real_feats"] = torch.FloatTensor(items.values[:, 2:])

        if os.path.exists(synthetic_data_path):
            self.impression_feats = torch.load(synthetic_data_path)
            self.impression_feats["labels"] = (
                self.impression_feats["label_probs"] >= cut).to(
                    dtype=torch.float32)
            print("loaded full_impression_feats.pt")
        else:
            print("generating impression_feats")
            self.simulator = ImpressionSimulator(use_impression_feats=True)
            self.simulator.load_state_dict(torch.load(simulator_path))
            self.simulator = self.simulator.to(device)

            impressions = self.get_full_impressions(ratings)

            self.impression_feats["user_ids"] = torch.LongTensor(
                impressions[:, 0])
            self.impression_feats["item_ids"] = torch.LongTensor(
                impressions[:, 1])
            self.impression_feats["real_feats"] = torch.FloatTensor(
                impressions[:, 2]).view(-1, 1)
            self.impression_feats["labels"] = torch.zeros_like(
                self.impression_feats["real_feats"])

            self.impression_feats["label_probs"] = self.generate_labels()
            self.impression_feats["labels"] = (
                self.impression_feats["label_probs"] >= cut).to(
                    dtype=torch.float32)

            torch.save(self.impression_feats, synthetic_data_path)
            print("saved impression_feats")

    def __len__(self):
        return len(self.impression_feats["user_ids"])

    def __getitem__(self, idx):
        labels = self.impression_feats["labels"][idx]
        feats = {}
        feats["impression_feats"] = {}
        feats["impression_feats"]["real_feats"] = self.impression_feats[
            "real_feats"][idx]
        user_id = self.impression_feats["user_ids"][idx]
        item_id = self.impression_feats["item_ids"][idx]
        feats["user_feats"] = {
            key: value[user_id - 1]
            for key, value in self.user_feats.items()
        }
        feats["item_feats"] = {
            key: value[item_id - 1]
            for key, value in self.item_feats.items()
        }
        return feats, labels

    def load_data(self, filepath):
        names = "UserID::MovieID::Rating::Timestamp".split("::")
        ratings = pd.read_csv(
            os.path.join(filepath, "ratings.dat"),
            sep="::",
            names=names,
            engine="python")
        ratings["Rating"] = (ratings["Rating"] > 3).astype(int)
        ratings["Timestamp"] = (
            ratings["Timestamp"] - ratings["Timestamp"].min()
        ) / float(ratings["Timestamp"].max() - ratings["Timestamp"].min())

        names = "UserID::Gender::Age::Occupation::Zip-code".split("::")
        users = pd.read_csv(
            os.path.join(filepath, "users.dat"),
            sep="::",
            names=names,
            engine="python")
        for i in range(1, users.shape[1]):
            users.iloc[:, i] = pd.factorize(users.iloc[:, i])[0]

        names = "MovieID::Title::Genres".split("::")
        Genres = [
            "Action", "Adventure", "Animation", "Children's", "Comedy",
            "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
            "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War",
            "Western"
        ]
        movies = pd.read_csv(
            os.path.join(filepath, "movies.dat"),
            sep="::",
            names=names,
            engine="python")
        movies["Year"] = movies["Title"].apply(lambda x: x[-5:-1])
        for genre in Genres:
            movies[genre] = movies["Genres"].apply(lambda x: genre in x)
        movies.iloc[:, 3] = pd.factorize(movies.iloc[:, 3])[0]
        movies.iloc[:, 4:] = movies.iloc[:, 4:].astype(float)
        movies = movies.loc[:, ["MovieID", "Year"] + Genres]
        movies.iloc[:, 2:] = movies.iloc[:, 2:].div(
            movies.iloc[:, 2:].sum(axis=1), axis=0)

        movie_id_map = {}
        for i in range(movies.shape[0]):
            movie_id_map[movies.loc[i, "MovieID"]] = i + 1

        movies["MovieID"] = movies["MovieID"].apply(lambda x: movie_id_map[x])
        ratings["MovieID"] = ratings["MovieID"].apply(
            lambda x: movie_id_map[x])

        self.NUM_ITEMS = len(movies.MovieID.unique())
        self.NUM_YEARS = len(movies.Year.unique())
        self.NUM_GENRES = movies.shape[1] - 2

        self.NUM_USERS = len(users.UserID.unique())
        self.NUM_OCCUPS = len(users.Occupation.unique())
        self.NUM_AGES = len(users.Age.unique())
        self.NUM_ZIPS = len(users["Zip-code"].unique())

        return ratings, users, movies

    def get_full_impressions(self, ratings):
        """Gets NUM_USERS x NUM_ITEMS impression features by iterating the user and item ids.
        The impression-level feature, i.e. the timestamp, is sampled from a normal distribution with
        mean and std as the empirical mean and std of each user's recorded timestamps in the real data.
        """
        timestamps = {}
        for i in range(len(ratings)):
            u_id = ratings.loc[i, "UserID"]
            timestamps[u_id] = timestamps.get(u_id, [])
            timestamps[u_id].append(ratings.loc[i, "Timestamp"])
        rs = np.random.RandomState(0)
        t_samples = []
        for i in range(self.NUM_USERS):
            u_id = i + 1
            t_samples.append(
                rs.normal(
                    loc=np.mean(timestamps[u_id]),
                    scale=np.std(timestamps[u_id]),
                    size=(self.NUM_ITEMS, )))
        t_samples = np.array(t_samples)

        impressions = []
        for i in range(self.NUM_USERS):
            for j in range(self.NUM_ITEMS):
                impressions.append([i + 1, j + 1, t_samples[i, j]])
        impressions = np.array(impressions)
        return impressions

    def to_device(self, data):
        if isinstance(data, torch.Tensor):
            return data.to(self.device)
        if isinstance(data, dict):
            transformed_data = {}
            for key in data:
                transformed_data[key] = self.to_device(data[key])
        elif type(data) == list:
            transformed_data = []
            for x in data:
                transformed_data.append(self.to_device(x))
        else:
            raise NotImplementedError(
                "Type {} not supported.".format(type(data)))
        return transformed_data

    def generate_labels(self):
        """Generates the binary labels using the simulator on every user-item pair."""
        with torch.no_grad():
            self.simulator.eval()
            preds = []
            for i in tqdm(range(len(self.impression_feats["labels"]) // 500)):
                feats, _ = self.__getitem__(
                    list(range(i * 500, (i + 1) * 500)))
                feats = self.to_device(feats)
                outputs = torch.sigmoid(self.simulator(**feats))
                preds += list(outputs.squeeze().cpu().numpy())
            if (i + 1) * 500 < len(self.impression_feats["labels"]):
                feats, _ = self.__getitem__(
                    list(
                        range((i + 1) * 500,
                              len(self.impression_feats["labels"]))))
                feats = self.to_device(feats)
                outputs = torch.sigmoid(self.simulator(**feats))
                preds += list(outputs.squeeze().cpu().numpy())
        return torch.FloatTensor(np.array(preds))

## Metrics

In [ ]:
class BaseMetric(object):
    def __init__(self, rel_threshold, k):
        self.rel_threshold = rel_threshold
        if np.isscalar(k):
            k = np.array([k])
        self.k = k

    def __len__(self):
        return len(self.k)

    def __call__(self, *args, **kwargs):
        raise NotImplementedError

    def _compute(self, *args, **kwargs):
        raise NotImplementedError

In [ ]:
class PrecisionRecall(BaseMetric):
    def __init__(self, rel_threshold=0, k=10):
        super(PrecisionRecall, self).__init__(rel_threshold, k)

    def __len__(self):
        return 2 * len(self.k)

    def __str__(self):
        str_precision = [('Precision@%1.f' % x) for x in self.k]
        str_recall = [('Recall@%1.f' % x) for x in self.k]
        return (','.join(str_precision)) + ',' + (','.join(str_recall))

    def __call__(self, targets, predictions):
        precision, recall = zip(
            *[self._compute(targets, predictions, x) for x in self.k])
        result = np.concatenate((precision, recall), axis=0)
        return result

    def _compute(self, targets, predictions, k):
        predictions = predictions[:k]
        num_hit = len(set(predictions).intersection(set(targets)))

        return float(num_hit) / len(predictions), float(num_hit) / len(targets)

In [ ]:
class MeanAP(BaseMetric):
    def __init__(self, rel_threshold=0, k=np.inf):
        super(MeanAP, self).__init__(rel_threshold, k)

    def __call__(self, targets, predictions):
        result = [self._compute(targets, predictions, x) for x in self.k]
        return np.array(result)

    def __str__(self):
        return ','.join([('MeanAP@%1.f' % x) for x in self.k])

    def _compute(self, targets, predictions, k):
        if len(predictions) > k:
            predictions = predictions[:k]

        score = 0.0
        num_hits = 0.0

        for i, p in enumerate(predictions):
            if p in targets and p not in predictions[:i]:
                num_hits += 1.0
                score += num_hits / (i + 1.0)

        if not list(targets):
            return 0.0

        return score / min(len(targets), k)

In [ ]:
class NormalizedDCG(BaseMetric):
    def __init__(self, rel_threshold=0, k=10):
        super(NormalizedDCG, self).__init__(rel_threshold, k)

    def __call__(self, targets, predictions):
        result = [self._compute(targets, predictions, x) for x in self.k]
        return np.array(result)

    def __str__(self):
        return ','.join([('NDCG@%1.f' % x) for x in self.k])

    def _compute(self, targets, predictions, k):
        k = min(len(targets), k)

        if len(predictions) > k:
            predictions = predictions[:k]

        # compute idcg
        idcg = np.sum(1 / np.log2(np.arange(2, k + 2)))
        dcg = 0.0
        for i, p in enumerate(predictions):
            if p in targets:
                dcg += 1 / np.log2(i + 2)
        ndcg = dcg / idcg

        return ndcg


all_metrics = [PrecisionRecall(k=[1, 5, 10]), NormalizedDCG(k=[5, 10, 20])]

In [ ]:
class Evaluator(object):
    """Evaluator for both one-stage and two-stage evaluations."""

    def __init__(self, u, a, simulator, syn):
        self.u = u
        self.a = a
        self.simulator = simulator
        self.syn = syn

        self.target_rankings = self.get_target_rankings()
        self.metrics = all_metrics

    def get_target_rankings(self):
        target_rankings = []
        with torch.no_grad():
            self.simulator.eval()
            for i in range(NUM_USERS):
                impression_ids = range(i * NUM_ITEMS, (i + 1) * NUM_ITEMS)
                feats, _ = self.syn[impression_ids]
                feats["impression_feats"]["real_feats"] = torch.mean(
                    feats["impression_feats"]["real_feats"],
                    dim=0,
                    keepdim=True).repeat([NUM_ITEMS, 1])
                feats = self.syn.to_device(feats)
                outputs = torch.sigmoid(self.simulator(**feats))
                user_target_ranking = (outputs >
                                       self.syn.cut).nonzero().view(-1)
                target_rankings.append(user_target_ranking.cpu().numpy())
        return target_rankings

    def one_stage_ranking_eval(self, logits, user_list):
        for i, user in enumerate(user_list):
            user_rated_items = self.a[self.u == user]
            logits[i, user_rated_items] = -np.inf
        sort_idx = torch.argsort(logits, dim=1, descending=True).cpu().numpy()
        # Init evaluation results.
        total_metrics_len = 0
        for metric in self.metrics:
            total_metrics_len += len(metric)

        total_val_metrics = np.zeros(
            [len(user_list), total_metrics_len], dtype=np.float32)
        valid_rows = []
        for i, user in enumerate(user_list):
            pred_ranking = sort_idx[i].tolist()
            target_ranking = self.target_rankings[user]
            if len(target_ranking) <= 0:
                continue
            metric_results = list()
            for j, metric in enumerate(self.metrics):
                result = metric(
                    targets=target_ranking, predictions=pred_ranking)
                metric_results.append(result)
            total_val_metrics[i, :] = np.concatenate(metric_results)
            valid_rows.append(i)
        # Average evaluation results by user.
        total_val_metrics = total_val_metrics[valid_rows]
        avg_val_metrics = (total_val_metrics.mean(axis=0)).tolist()
        # Summary evaluation results into a dict.
        ind, result = 0, OrderedDict()
        for metric in self.metrics:
            values = avg_val_metrics[ind:ind + len(metric)]
            if len(values) <= 1:
                result[str(metric)] = values
            else:
                for name, value in zip(str(metric).split(','), values):
                    result[name] = value
            ind += len(metric)
        return result

    def two_stage_ranking_eval(self, logits, ranker, user_list, k=30):
        sort_idx = torch.argsort(logits, dim=1, descending=True).cpu().numpy()
        topk_item_ids = []
        for i, user in enumerate(user_list):
            topk_item_ids.append([])
            for j in sort_idx[i]:
                if j not in self.a[self.u == user]:
                    topk_item_ids[-1].append(j)
                if len(topk_item_ids[-1]) == k:
                    break
        time_feats = self.syn.to_device(
            torch.mean(
                self.syn.impression_feats["real_feats"].view(
                    NUM_USERS, NUM_ITEMS),
                dim=1).view(-1, 1))
        # Init evaluation results.
        total_metrics_len = 0
        for metric in self.metrics:
            total_metrics_len += len(metric)

        total_val_metrics = np.zeros(
            [len(user_list), total_metrics_len], dtype=np.float32)
        valid_rows = []
        for i, user in enumerate(user_list):
            user_feats = {
                key: value[user].view(1, -1)
                for key, value in self.syn.user_feats.items()
            }
            item_feats = {
                key: value[topk_item_ids[i]]
                for key, value in self.syn.item_feats.items()
            }

            user_feats = self.syn.to_device(user_feats)
            item_feats = self.syn.to_device(item_feats)
            impression_feats = {"real_feats": time_feats[user].view(1, -1)}
            ranker_logits = ranker(user_feats, item_feats,
                                   impression_feats).view(1, -1)
            _, pred = ranker_logits.topk(k=k)
            pred = pred[0].cpu().numpy()
            pred_ranking = sort_idx[i][pred].tolist()
            target_ranking = self.target_rankings[user]
            if len(target_ranking) <= 0:
                continue
            metric_results = list()
            for j, metric in enumerate(self.metrics):
                result = metric(
                    targets=target_ranking, predictions=pred_ranking)
                metric_results.append(result)
            total_val_metrics[i, :] = np.concatenate(metric_results)
            valid_rows.append(i)
            # Average evaluation results by user.
        total_val_metrics = total_val_metrics[valid_rows]
        avg_val_metrics = (total_val_metrics.mean(axis=0)).tolist()
        # Summary evaluation results into a dict.
        ind, result = 0, OrderedDict()
        for metric in self.metrics:
            values = avg_val_metrics[ind:ind + len(metric)]
            if len(values) <= 1:
                result[str(metric)] = values
            else:
                for name, value in zip(str(metric).split(','), values):
                    result[name] = value
            ind += len(metric)
        return result

    def one_stage_eval(self, logits):
        sort_idx = torch.argsort(logits, dim=1, descending=True).cpu().numpy()
        impression_ids = []
        for i in range(NUM_USERS):
            for j in sort_idx[i]:
                if j not in self.a[self.u == i]:
                    break
            impression_ids.append(i * NUM_ITEMS + j)
        feats, labels = self.syn[impression_ids]
        feats["impression_feats"]["real_feats"] = torch.mean(
            self.syn.impression_feats["real_feats"].view(NUM_USERS, NUM_ITEMS),
            dim=1).view(-1, 1)
        with torch.no_grad():
            self.simulator.eval()
            feats = self.syn.to_device(feats)
            outputs = torch.sigmoid(self.simulator(**feats))
            return torch.mean(
                (outputs > self.syn.cut).to(dtype=torch.float32)).item()

    def two_stage_eval(self, logits, ranker, k=30):
        sort_idx = torch.argsort(logits, dim=1, descending=True).cpu().numpy()
        topk_item_ids = []
        for i in range(NUM_USERS):
            topk_item_ids.append([])
            for j in sort_idx[i]:
                if j not in self.a[self.u == i]:
                    topk_item_ids[-1].append(j)
                if len(topk_item_ids[-1]) == k:
                    break
        time_feats = self.syn.to_device(
            torch.mean(
                self.syn.impression_feats["real_feats"].view(
                    NUM_USERS, NUM_ITEMS),
                dim=1).view(-1, 1))
        recommneded = []
        for i in range(NUM_USERS):
            user_feats = {
                key: value[i].view(1, -1)
                for key, value in self.syn.user_feats.items()
            }
            item_feats = {
                key: value[topk_item_ids[i]]
                for key, value in self.syn.item_feats.items()
            }

            user_feats = self.syn.to_device(user_feats)
            item_feats = self.syn.to_device(item_feats)
            impression_feats = {"real_feats": time_feats[i].view(1, -1)}
            ranker_logits = ranker(user_feats, item_feats,
                                   impression_feats).view(1, -1)
            _, pred = torch.max(ranker_logits, 1)
            pred = pred.squeeze().item()
            recommneded.append(topk_item_ids[i][pred])

        impression_ids = []
        for i in range(NUM_USERS):
            impression_ids.append(i * NUM_ITEMS + recommneded[i])
        feats, labels = self.syn[impression_ids]
        feats["impression_feats"]["real_feats"] = torch.mean(
            self.syn.impression_feats["real_feats"].view(NUM_USERS, NUM_ITEMS),
            dim=1).view(-1, 1)
        with torch.no_grad():
            self.simulator.eval()
            feats = self.syn.to_device(feats)
            outputs = torch.sigmoid(self.simulator(**feats))
            return torch.mean(
                (outputs > self.syn.cut).to(dtype=torch.float32)).item()

## Losses

In [ ]:
def batch_select(mat, idx):
    mask = torch.arange(mat.size(1)).expand_as(mat).to(
        mat.device, dtype=torch.long)
    mask = (mask == idx.view(-1, 1))
    return torch.masked_select(mat, mask)


def unique_and_padding(mat, padding_idx, dim=-1):
    """Conducts unique operation along dim and pads to the same length."""
    samples, _ = torch.sort(mat, dim=dim)
    samples_roll = torch.roll(samples, -1, dims=dim)
    samples_diff = samples - samples_roll
    samples_diff[:,
                 -1] = 1  # deal with the edge case that there is only one unique sample in a row
    samples_mask = torch.bitwise_not(samples_diff == 0)  # unique mask
    samples *= samples_mask.to(dtype=samples.dtype)
    samples += (1 - samples_mask.to(dtype=samples.dtype)) * padding_idx
    samples, _ = torch.sort(samples, dim=dim)
    # shrink size to max unique length
    samples = torch.unique(samples, dim=dim)
    return samples

In [ ]:
def loss_ce(logits, a, unused_p=None, unused_ranker_logits=None):
    """Cross entropy."""
    return -torch.mean(batch_select(F.log_softmax(logits, dim=1), a))

In [ ]:
def loss_ips(logits,
             a,
             p,
             unused_ranker_logits=None,
             upper_limit=100,
             lower_limit=0.01):
    """IPS loss (one-stage)."""
    importance_weight = batch_select(F.softmax(logits.detach(), dim=1), a) / p
    importance_weight = torch.where(
        importance_weight > lower_limit, importance_weight,
        lower_limit * torch.ones_like(importance_weight))
    importance_weight = torch.where(
        importance_weight < upper_limit, importance_weight,
        upper_limit * torch.ones_like(importance_weight))
    importance_weight /= torch.mean(importance_weight)
    return -torch.mean(
        batch_select(F.log_softmax(logits, dim=1), a) * importance_weight)

In [ ]:
def loss_2s(logits,
            a,
            p,
            ranker_logits,
            slate_sample_size=100,
            slate_size=30,
            temperature=np.e,
            alpha=1e-5,
            upper_limit=100,
            lower_limit=0.01):
    """Two stage loss."""
    num_logits = logits.size(1)
    rls = ranker_logits.detach()
    probs = F.softmax(logits.detach() / temperature, dim=1)
    log_probs = F.log_softmax(logits, dim=1)

    rls = torch.cat(
        [
            rls,
            torch.Tensor([float("-inf")]).to(rls.device).view(1, 1).expand(
                rls.size(0), 1)
        ],
        dim=1)
    log_probs = torch.cat(
        [log_probs,
         torch.zeros(log_probs.size(0), 1).to(log_probs.device)],
        dim=1)

    importance_weight = batch_select(F.softmax(logits.detach(), dim=1), a) / p
    importance_weight = torch.where(
        importance_weight > lower_limit, importance_weight,
        lower_limit * torch.ones_like(importance_weight))
    importance_weight = torch.where(
        importance_weight < upper_limit, importance_weight,
        upper_limit * torch.ones_like(importance_weight))
    importance_weight /= torch.mean(importance_weight)

    log_action_res = []
    sampled_slate_res = []
    for i in range(probs.size(0)):
        samples = torch.multinomial(
            probs[i], slate_sample_size * slate_size, replacement=True).view(
                slate_sample_size, slate_size)
        samples = torch.cat(
            [samples, a[i].view(1, 1).expand(samples.size(0), 1)], dim=1)
        samples = unique_and_padding(samples, num_logits)
        rp = F.softmax(
            F.embedding(samples, rls[i].view(-1, 1)).squeeze(-1), dim=1)
        lp = torch.sum(
            F.embedding(samples, log_probs[i].view(-1, 1)).squeeze(-1),
            dim=1) - log_probs[i, a[i]]
        sampled_slate_res.append(
            torch.mean(importance_weight[i] * rp[samples == a[i]] * lp))
        log_action_res.append(
            torch.mean(importance_weight[i] * rp[samples == a[i]]))
    loss = -torch.mean(
        torch.stack(log_action_res) * batch_select(log_probs, a))
    loss += -torch.mean(torch.stack(sampled_slate_res)) * alpha
    return loss

## Main

In [ ]:
!pip install -q torchnet

In [ ]:
import argparse
import os

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from six.moves import cPickle as pickle
from torch.utils.data import DataLoader, Subset
from torchnet.meter import AUCMeter

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--verbose", type=int, default=1, help="Verbose.")
parser.add_argument("--seed", type=int, default=0, help="Random seed.")
parser.add_argument("--loss_type", default="loss_ce")
parser.add_argument("--device", default="cuda:0")
parser.add_argument("--alpha", type=float, default=1e-3, help="Loss ratio.")
parser.add_argument("--lr", type=float, default=0.05, help="Learning rate.")
args = parser.parse_args(args={})

torch.manual_seed(0)
torch.cuda.manual_seed(0)

filepath = "./ml-1m"
device = args.device

dataset = MovieLensDataset(filepath, device=device)

NUM_ITEMS = dataset.NUM_ITEMS
NUM_YEARS = dataset.NUM_YEARS
NUM_GENRES = dataset.NUM_GENRES

NUM_USERS = dataset.NUM_USERS
NUM_OCCUPS = dataset.NUM_OCCUPS
NUM_AGES = dataset.NUM_AGES
NUM_ZIPS = dataset.NUM_ZIPS

In [ ]:
simulator_path = os.path.join(filepath, "simulator.pt")
if os.path.exists(simulator_path):
    simulator = ImpressionSimulator(use_impression_feats=True)
    simulator.load_state_dict(torch.load(simulator_path))
    simulator.to(device)
    simulator.eval()
else:
    # train a simulator model on the original ML-1M dataset
    # the simulator will be used to generate synthetic labels later

    torch.manual_seed(0)
    torch.cuda.manual_seed(0)

    num_samples = len(dataset)
    train_loader = DataLoader(
        Subset(dataset, list(range(num_samples * 3 // 5))),
        batch_size=128,
        shuffle=True)
    val_loader = DataLoader(
        Subset(dataset,
               list(range(num_samples * 3 // 5, num_samples * 4 // 5))),
        batch_size=128)
    test_loader = DataLoader(
        Subset(dataset, list(range(num_samples * 4 // 5, num_samples))),
        batch_size=128)

    simulator = ImpressionSimulator(use_impression_feats=True).to(device)
    opt = torch.optim.Adagrad(
        simulator.parameters(), lr=0.05, weight_decay=1e-4)
    criterion = nn.BCEWithLogitsLoss()

    simulator.train()
    for epoch in range(4):
        print("---epoch {}---".format(epoch))
        for step, batch in enumerate(train_loader):
            feats, labels = batch
            logits = simulator(**feats)
            loss = criterion(logits, labels)

            opt.zero_grad()
            loss.backward()
            opt.step()

            if (step + 1) % 500 == 0:
                with torch.no_grad():
                    simulator.eval()
                    auc = AUCMeter()
                    for feats, labels in val_loader:
                        outputs = torch.sigmoid(simulator(**feats))
                        auc.add(outputs, labels)
                    print(step, auc.value()[0])
                    if auc.value()[0] > 0.735:
                        break
                simulator.train()

    simulator.to("cpu")
    torch.save(simulator.state_dict(), simulator_path)

---epoch 0---
499 0.6932387523620361
999 0.7135402368767171
1499 0.7215369956250887
1999 0.7251487885185414
2499 0.7250320801426405
2999 0.7272670074151073
3499 0.7281266442311184
3999 0.7294850286875231
4499 0.7296389614963857
---epoch 1---
499 0.7283547770191316
999 0.7295152100721607
1499 0.7300909216769196
1999 0.7302949709078976
2499 0.7307115513456678
2999 0.7303182560548731
3499 0.729832712732132
3999 0.7313203021611654
4499 0.7318606932964403
---epoch 2---
499 0.7313658487455917
999 0.731270933721071
1499 0.7307394064320014
1999 0.7323340285603053
2499 0.7322074757365146
2999 0.7321213073333425
3499 0.7330514084885155
3999 0.7328071902744475
4499 0.7332603719115534
---epoch 3---
499 0.7324623727995114
999 0.7323136933519381
1499 0.7327187807719571
1999 0.7329365712368654
2499 0.7318917432430467
2999 0.733582752772605
3499 0.7342934273983486
3999 0.7341365564445322
4499 0.7340031962421637


In [ ]:
# create a torch dataset class that adopt the simulator and generate the synthetic dataset
synthetic_data_path = os.path.join(filepath, "full_impression_feats.pt")
syn = SyntheticMovieLensDataset(
    filepath, simulator_path, synthetic_data_path, device=device)

logging_policy_path = os.path.join(filepath, "logging_policy.pt")
if os.path.exists(logging_policy_path):
    logging_policy = Nominator()
    logging_policy.load_state_dict(torch.load(logging_policy_path))
    logging_policy.to(device)
    logging_policy.eval()
else:
    # train a logging policy using the synthetic dataset
    num_samples = len(syn)
    idx_list = list(range(num_samples))
    rs = np.random.RandomState(0)
    rs.shuffle(idx_list)
    train_idx = idx_list[:10000]
    val_idx = idx_list[10000:20000]
    test_idx = idx_list[-100000:]
    train_loader = DataLoader(
        Subset(syn, train_idx), batch_size=128, shuffle=True)
    val_loader = DataLoader(Subset(syn, val_idx), batch_size=128)
    test_loader = DataLoader(Subset(syn, test_idx), batch_size=128)

    logging_policy = Nominator().to(device)
    opt = torch.optim.Adagrad(
        logging_policy.parameters(), lr=0.05, weight_decay=1e-4)
    criterion = nn.BCEWithLogitsLoss()

    logging_policy.train()
    for epoch in range(40):
        print("---epoch {}---".format(epoch))
        for step, batch in enumerate(train_loader):
            feats, labels = batch
            feats = syn.to_device(feats)
            labels = syn.to_device(labels)
            logits = logging_policy(feats["user_feats"], feats["item_feats"])
            loss = criterion(logits, labels)

            opt.zero_grad()
            loss.backward()
            opt.step()

        with torch.no_grad():
            logging_policy.eval()
            auc = AUCMeter()
            for feats, labels in val_loader:
                feats = syn.to_device(feats)
                labels = syn.to_device(labels)
                outputs = torch.sigmoid(
                    logging_policy(feats["user_feats"], feats["item_feats"]))
                auc.add(outputs, labels)
            print(step, auc.value()[0])

            logging_policy.train()

    logging_policy.eval()

    logging_policy.to("cpu")
    torch.save(logging_policy.state_dict(), logging_policy_path)
    logging_policy.to(device)

generating impression_feats


100%|██████████| 46906/46906 [01:04<00:00, 726.33it/s]


saved impression_feats
---epoch 0---
78 0.7303382306338263
---epoch 1---
78 0.7581630234600971
---epoch 2---
78 0.7758875382286556
---epoch 3---
78 0.7947954816858157
---epoch 4---
78 0.8092959307857327
---epoch 5---
78 0.8209026242576345
---epoch 6---
78 0.8281561300331621
---epoch 7---
78 0.831641647559472
---epoch 8---
78 0.8359850391331326
---epoch 9---
78 0.838561799838773
---epoch 10---
78 0.8401704563395371
---epoch 11---
78 0.8411352317648504
---epoch 12---
78 0.8419039896279034
---epoch 13---
78 0.8420276300116679
---epoch 14---
78 0.8417600912427988
---epoch 15---
78 0.8424424910531916
---epoch 16---
78 0.8427186846489241
---epoch 17---
78 0.8426326119202265
---epoch 18---
78 0.8428287245904745
---epoch 19---
78 0.842670274683281
---epoch 20---
78 0.8428754226123425
---epoch 21---
78 0.8429700550599161
---epoch 22---
78 0.8431265076993719
---epoch 23---
78 0.8433620901844372
---epoch 24---
78 0.8435445073044836
---epoch 25---
78 0.8435860694950261
---epoch 26---
78 0.84358293

In [ ]:
def generate_bandit_samples(logging_policy, syn, k=5):
    """Generates partial-labeled bandit samples with the logging policy.
    Arguments:
        k: The number of items to be sampled for each user.
    """
    logging_policy.set_binary(False)
    with torch.no_grad():
        feats = {}
        feats["user_feats"] = syn.user_feats
        feats["item_feats"] = syn.item_feats
        feats = syn.to_device(feats)
        probs = F.softmax(logging_policy(**feats), dim=1)

    sampled_users = []
    sampled_actions = []
    sampled_probs = []
    sampled_rewards = []
    for i in range(probs.size(0)):
        sampled_users.append([i] * k)
        sampled_actions.append(
            torch.multinomial(probs[i], k).cpu().numpy().tolist())
        sampled_probs.append(
            probs[i, sampled_actions[-1]].cpu().numpy().tolist())
        sampled_rewards.append(syn.impression_feats["labels"][[
            i * probs.size(1) + j for j in sampled_actions[-1]
        ]].numpy().tolist())
    return np.array(sampled_users).reshape(-1), np.array(
        sampled_actions).reshape(-1), np.array(sampled_probs).reshape(
            -1), np.array(sampled_rewards).reshape(-1)


torch.manual_seed(0)
torch.cuda.manual_seed(0)

u, a, p, r = generate_bandit_samples(
    logging_policy, syn,
    k=5)  # u: user, a: item, p: logging policy probability, r: reward/label

simulator = simulator.to(device)
ev = Evaluator(u[r > 0], a[r > 0], simulator, syn)

all_user_feats = syn.to_device(syn.user_feats)
all_item_feats = syn.to_device(syn.item_feats)
all_impression_feats = syn.to_device({
    "real_feats":
    torch.mean(
        syn.impression_feats["real_feats"].view(NUM_USERS, NUM_ITEMS),
        dim=1).view(-1, 1)
})

In [ ]:
# Split validation/test users.
num_val_users = 2000
val_user_list = list(range(0, num_val_users))
test_user_list = list(range(num_val_users, NUM_USERS))

test_item_feats = all_item_feats
test_user_feats = syn.to_device(
    {key: value[test_user_list]
     for key, value in all_user_feats.items()})
test_impression_feats = syn.to_device({
    key: value[test_user_list]
    for key, value in all_impression_feats.items()
})

val_item_feats = all_item_feats
val_user_feats = syn.to_device(
    {key: value[val_user_list]
     for key, value in all_user_feats.items()})
val_impression_feats = syn.to_device(
    {key: value[val_user_list]
     for key, value in all_impression_feats.items()})

ranker_path = os.path.join(filepath, "ranker.pt")

if os.path.exists(ranker_path):
    ranker = Ranker()
    ranker.load_state_dict(torch.load(ranker_path))
    ranker.to(device)
    ranker.eval()
    ranker.set_binary(False)
else:
    # train the ranker with binary cross-entropy
    torch.manual_seed(0)
    torch.cuda.manual_seed(0)

    batch_size = 128
    neg_sample_size = 29

    ranker = Ranker().to(device)
    opt = torch.optim.Adagrad(ranker.parameters(), lr=0.05, weight_decay=1e-4)
    criterion = nn.BCEWithLogitsLoss()

    rs = np.random.RandomState(0)
    ranker.train()
    for epoch in range(10):
        print("---epoch {}---".format(epoch))
        for step in range(len(u) // batch_size):
            user_list = u[step * batch_size:(step + 1) * batch_size]
            item_list = a[step * batch_size:(step + 1) * batch_size]
            user_feats = syn.to_device({
                key: value[user_list]
                for key, value in syn.user_feats.items()
            })
            item_feats = syn.to_device({
                key: value[item_list]
                for key, value in syn.item_feats.items()
            })
            impression_list = [
                user_id * NUM_ITEMS + item_id
                for user_id, item_id in zip(user_list, item_list)
            ]
            impression_feats = syn.to_device({
                "real_feats":
                syn.impression_feats["real_feats"][impression_list]
            })

            labels = torch.FloatTensor(
                r[step * batch_size:(step + 1) * batch_size]).to(device)

            logits = ranker(user_feats, item_feats, impression_feats)
            loss = criterion(logits, labels)

            opt.zero_grad()
            loss.backward()
            opt.step()

        with torch.no_grad():
            ranker.eval()
            ranker.set_binary(False)
            logits = ranker(all_user_feats, all_item_feats,
                            all_impression_feats)

            print(step, ev.one_stage_eval(logits))

            # Evaluate ranking metrics on validation users.
            logits = ranker(val_user_feats, val_item_feats,
                            val_impression_feats)
            print(step, ev.one_stage_ranking_eval(logits, val_user_list))

            ranker.train()
            ranker.set_binary(True)

    ranker.eval()
    ranker.set_binary(False)

    ranker.to("cpu")
    torch.save(ranker.state_dict(), ranker_path)
    ranker.to(device)

---epoch 0---
234 0.49751657247543335
234 OrderedDict([('Precision@1', 0.525853157043457), ('Precision@5', 0.37435382604599), ('Precision@10', 0.3275078535079956), ('Recall@1', 0.001950422883965075), ('Recall@5', 0.006817308254539967), ('Recall@10', 0.011337128467857838), ('NDCG@5', 0.40602830052375793), ('NDCG@10', 0.36229410767555237), ('NDCG@20', 0.3150430917739868)])
---epoch 1---
234 0.5798013210296631
234 OrderedDict([('Precision@1', 0.6003102660179138), ('Precision@5', 0.4798329174518585), ('Precision@10', 0.42228519916534424), ('Recall@1', 0.002812947379425168), ('Recall@5', 0.011755581945180893), ('Recall@10', 0.02120870165526867), ('NDCG@5', 0.5004847645759583), ('NDCG@10', 0.45322883129119873), ('NDCG@20', 0.39284855127334595)])
---epoch 2---
234 0.5889073014259338
234 OrderedDict([('Precision@1', 0.6106514930725098), ('Precision@5', 0.5765244960784912), ('Precision@10', 0.5148389339447021), ('Recall@1', 0.003235821146517992), ('Recall@5', 0.016686489805579185), ('Recall@10'

In [ ]:
u = u[r > 0]
a = a[r > 0]
p = p[r > 0]

batch_size = 128
check_metric = "Precision@10"

torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

nominator = Nominator().to(device)
nominator.set_binary(False)

opt = torch.optim.Adagrad(
    nominator.parameters(), lr=args.lr, weight_decay=1e-4)

rs = np.random.RandomState(0)
nominator.train()

Nominator(
  (item_rep): ItemRep(
    (item_embedding): Embedding(3884, 10, padding_idx=0)
    (year_embedding): Embedding(81, 5)
    (genre_linear): Linear(in_features=18, out_features=5, bias=True)
  )
  (user_rep): UserRep(
    (user_embedding): Embedding(6041, 10, padding_idx=0)
    (gender_embedding): Embedding(2, 5)
    (age_embedding): Embedding(7, 5)
    (occup_embedding): Embedding(21, 5)
    (zip_embedding): Embedding(3439, 5)
  )
  (linear): Linear(in_features=30, out_features=20, bias=True)
)

In [ ]:
# Init results.
best_epoch = 0
best_result = 0.0
val_results, test_results = [], []

if args.loss_type == "loss_2s":
    with torch.no_grad():
        ranker.eval()
        ranker.set_binary(False)
        ranker_logits = ranker(all_user_feats, all_item_feats,
                               all_impression_feats)

for epoch in range(20):
    print("---epoch {}---".format(epoch))
    for step in range(len(u) // batch_size):
        item_ids = torch.LongTensor(
            a[step * batch_size:(step + 1) * batch_size]).to(device)
        item_probs = torch.FloatTensor(
            p[step * batch_size:(step + 1) * batch_size]).to(device)

        user_ids = u[step * batch_size:(step + 1) * batch_size]
        user_feats = {
            key: value[user_ids]
            for key, value in syn.user_feats.items()
        }
        user_feats = syn.to_device(user_feats)

        logits = nominator(user_feats, val_item_feats)

        if args.loss_type == "loss_ce":
            loss = loss_ce(logits, item_ids, item_probs)
        elif args.loss_type == "loss_ips":
            loss = loss_ips(logits, item_ids, item_probs, upper_limit=10)
        elif args.loss_type == "loss_2s":
            batch_ranker_logits = F.embedding(
                torch.LongTensor(user_ids).to(device), ranker_logits)
            loss = loss_2s(
                logits,
                item_ids,
                item_probs,
                batch_ranker_logits,
                upper_limit=10,
                alpha=args.alpha)
        else:
            raise NotImplementedError(
                "{} not supported.".format(args.loss_type))

        opt.zero_grad()
        loss.backward()
        opt.step()

    with torch.no_grad():
        nominator.eval()

        logits = nominator(all_user_feats, all_item_feats)
        print("1 stage", ev.one_stage_eval(logits))
        print("2 stage", ev.two_stage_eval(logits, ranker))

        # Evaluate ranking metrics on validation users.
        logits = nominator(val_user_feats, val_item_feats)
        one_stage_results = ev.one_stage_ranking_eval(logits, val_user_list)
        print("1 stage (val)", one_stage_results)
        two_stage_results = ev.two_stage_ranking_eval(logits, ranker,
                                                      val_user_list)
        print("2 stage (val)", two_stage_results)
        val_results.append((one_stage_results, two_stage_results))
        # Log best epoch
        if two_stage_results[check_metric] > best_result:
            best_epoch = epoch
            best_result = two_stage_results[check_metric]
        # Evaluate ranking metrics on test users.
        logits = nominator(test_user_feats, test_item_feats)
        one_stage_results = ev.one_stage_ranking_eval(logits, test_user_list)
        print("1 stage (test)", one_stage_results)
        two_stage_results = ev.two_stage_ranking_eval(logits, ranker,
                                                      test_user_list)
        print("2 stage (test)", two_stage_results)
        test_results.append((one_stage_results, two_stage_results))

        nominator.train()

---epoch 0---
1 stage 0.8389073014259338
2 stage 0.931456983089447
1 stage (val) OrderedDict([('Precision@1', 0.7761116623878479), ('Precision@5', 0.7825230956077576), ('Precision@10', 0.7693895101547241), ('Recall@1', 0.0026108406018465757), ('Recall@5', 0.01443537138402462), ('Recall@10', 0.02800210937857628), ('NDCG@5', 0.7805129289627075), ('NDCG@10', 0.7720701694488525), ('NDCG@20', 0.7773204445838928)])
2 stage (val) OrderedDict([('Precision@1', 0.9152016639709473), ('Precision@5', 0.8799382448196411), ('Precision@10', 0.8639603853225708), ('Recall@1', 0.010427681729197502), ('Recall@5', 0.02969161979854107), ('Recall@10', 0.04909321665763855), ('NDCG@5', 0.8888923525810242), ('NDCG@10', 0.8768721222877502), ('NDCG@20', 0.8476952910423279)])
1 stage (test) OrderedDict([('Precision@1', 0.894406795501709), ('Precision@5', 0.8917496204376221), ('Precision@10', 0.8819892406463623), ('Recall@1', 0.003005631733685732), ('Recall@5', 0.015431685373187065), ('Recall@10', 0.030124178156256

In [ ]:
print("Best validation epoch: {}".format(best_epoch))
print("Best validation stage results\n 1 stage: {}\n 2 stage: {}".format(
    val_results[best_epoch][0], val_results[best_epoch][1]))
print("Best test results\n 1 stage: {}\n 2 stage: {}".format(
    test_results[best_epoch][0], test_results[best_epoch][1]))

Best validation epoch: 6
Best validation stage results
 1 stage: OrderedDict([('Precision@1', 0.7652533650398254), ('Precision@5', 0.7777659296989441), ('Precision@10', 0.7713546752929688), ('Recall@1', 0.002462433883920312), ('Recall@5', 0.014112057164311409), ('Recall@10', 0.02813573367893696), ('NDCG@5', 0.7751555442810059), ('NDCG@10', 0.7717844247817993), ('NDCG@20', 0.7759582996368408)])
 2 stage: OrderedDict([('Precision@1', 0.9353671073913574), ('Precision@5', 0.8887278437614441), ('Precision@10', 0.866390585899353), ('Recall@1', 0.013995282351970673), ('Recall@5', 0.035590916872024536), ('Recall@10', 0.05474210903048515), ('NDCG@5', 0.9002852439880371), ('NDCG@10', 0.8846437931060791), ('NDCG@20', 0.8533338904380798)])
Best test results
 1 stage: OrderedDict([('Precision@1', 0.8931527733802795), ('Precision@5', 0.8867837190628052), ('Precision@10', 0.8830179572105408), ('Recall@1', 0.0030013115610927343), ('Recall@5', 0.01514824852347374), ('Recall@10', 0.03022931143641472), (

In [ ]:
pickle.dump((best_epoch, val_results, test_results),
            open("{}-a{}_{}.pkl".format(
                args.loss_type.split("_")[1], args.alpha, args.seed), "wb"))

In [ ]:
!apt-get install tree

In [ ]:
!tree --du -h

.
├── [ 20K]  ce-a0.001_0.pkl
├── [651M]  ml-1m
│   ├── [626M]  full_impression_feats.pt
│   ├── [463K]  logging_policy.pt
│   ├── [167K]  movies.dat
│   ├── [463K]  ranker.pt
│   ├── [ 23M]  ratings.dat
│   ├── [5.4K]  README
│   ├── [502K]  simulator.pt
│   └── [131K]  users.dat
├── [5.6M]  ml-1m.zip
└── [ 54M]  sample_data
    ├── [1.7K]  anscombe.json
    ├── [294K]  california_housing_test.csv
    ├── [1.6M]  california_housing_train.csv
    ├── [ 17M]  mnist_test.csv
    ├── [ 35M]  mnist_train_small.csv
    └── [ 930]  README.md

 711M used in 2 directories, 16 files


## Experimental results

![](https://github.com/recohut/drl-recsys/raw/a8a38a68e3606557f3501c8adc85fc6aa5726bc1/docs/_images/T257798_4.png)

## Conclusion

- The results of 2-IPS method in both evaluations are better than 1-IPS and cross-entropy.
- 1-IPS performs better than the cross-entropy method in one-stage evaluation, and performs worse than the cross-entropy method in two-stage evaluation, indicating that only improving the performance of a part of the system may not necessarily improve the performance of the entire system.